### Links
> Analise de redes


In [1]:
import $ivy.`org.apache.spark::spark-core:3.1.2`
import $ivy.`org.apache.spark::spark-sql:3.1.2`
import $ivy.`org.apache.hadoop:hadoop-client:3.1.2`
import $ivy.`org.apache.hadoop:hadoop-aws:3.1.2`
import $ivy.`com.amazonaws:aws-java-sdk-s3:1.11.375`
import $ivy.`com.amazonaws:aws-java-sdk-core:1.11.375`


import $ivy.$                                   

import $ivy.$                                  

import $ivy.$                                      

import $ivy.$                                   

import $ivy.$                                       

import $ivy.$                                         


In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
    import org.apache.log4j.{Level, Logger}
    Logger.getLogger("org").setLevel(Level.OFF)
      SparkSession.builder()
        .master("local[3]")
        .config("spark.sql.warehouse.dir", "s3://fernando-bontorin-censo")
        .getOrCreate()
}

import spark.implicits._
import org.apache.spark.sql.expressions.Window

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@76b1c199
import spark.implicits._

import org.apache.spark.sql.expressions.Window

In [3]:
val estabs = spark.read.options(
    Map("inferSchema" -> "true", "header" -> "true", "delimiter" -> ",", "encoding" -> "UTF-8")
).csv("s3a://fernando-bontorin-datasus/tbEstabelecimentoFull")

estabs.printSchema

root
 |-- CO_UNIDADE: long (nullable = true)
 |-- CO_CNES: integer (nullable = true)
 |-- NU_CNPJ_MANTENEDORA: long (nullable = true)
 |-- TP_PFPJ: integer (nullable = true)
 |-- NIVEL_DEP: integer (nullable = true)
 |-- NO_RAZAO_SOCIAL: string (nullable = true)
 |-- NO_FANTASIA: string (nullable = true)
 |-- NO_LOGRADOURO: string (nullable = true)
 |-- NU_ENDERECO: string (nullable = true)
 |-- NO_COMPLEMENTO: string (nullable = true)
 |-- NO_BAIRRO: string (nullable = true)
 |-- CO_CEP: integer (nullable = true)
 |-- NU_CPF: string (nullable = true)
 |-- NU_CNPJ: long (nullable = true)
 |-- CO_ATIVIDADE: integer (nullable = true)
 |-- CO_CLIENTELA: integer (nullable = true)
 |-- TP_UNIDADE: integer (nullable = true)
 |-- CO_ESTADO_GESTOR: integer (nullable = true)
 |-- CO_MUNICIPIO_GESTOR: integer (nullable = true)
 |-- TP_ESTAB_SEMPRE_ABERTO: string (nullable = true)
 |-- CO_TIPO_UNIDADE: string (nullable = true)
 |-- NO_FANTASIA_ABREV: string (nullable = true)
 |-- TP_GESTAO: strin

estabs: DataFrame = [CO_UNIDADE: bigint, CO_CNES: int ... 24 more fields]

In [4]:
val redes = estabs
    .where($"competencia" === "2021-12-01")
    .select("CO_UNIDADE", "NO_RAZAO_SOCIAL")
    .groupBy("NO_RAZAO_SOCIAL").count

redes: DataFrame = [NO_RAZAO_SOCIAL: string, count: bigint]

In [5]:
redes.orderBy(col("count").desc).show(false)

+------------------------------------------------+-----+
|NO_RAZAO_SOCIAL                                 |count|
+------------------------------------------------+-----+
|RAIA DROGASIL S A                               |199  |
|EMPREENDIMENTOS PAGUE MENOS S A                 |63   |
|DROGARIA SAO PAULO S A                          |60   |
|SECRETARIA DE ESTADO DA SAUDE DE SAO PAULO      |19   |
|FUNDACAO OSWALDO CRUZ                           |8    |
|DROGARIA ONOFRE LTDA                            |7    |
|EMPREENDIMENTOS PAGUE MENOS SA                  |6    |
|IMIFARMA PRODUTOS FARMACEUTICOS E COSMETICOS S A|6    |
|DIMED S A DISTRIBUIDORA DE MEDICAMENTOS         |5    |
|DROGARIA SAO PAULO                              |5    |
|EMPREENDIMENTOS PAGUE MENOS                     |4    |
|CIP FARMACEUTICA LTDA                           |2    |
|EMPREENDIMENTOS PAGUE MENOS S A 216             |2    |
|DROGARIA SAUDE E BELEZA POPULAR LTDA            |2    |
|CARREFOUR COMERCIO E INDUSTRIA

In [6]:
redes.orderBy(col("count").asc).show(false)

+------------------------------------------------+-----+
|NO_RAZAO_SOCIAL                                 |count|
+------------------------------------------------+-----+
|RAIA DROGASIL SA                                |1    |
|HEMOMED INSTITUTO DE ONCOLOGIA E HEMATOLOGIA    |1    |
|AROMATICI COMERCIO DE PRODUTOS MANIPULADOS LTDA |1    |
|FARMACIA BOTANICA A NATUREZA LTDA ME            |1    |
|4BIO MEDICAMENTOS SA                            |1    |
|FARMACIA ABDS EIRELI                            |1    |
|DROGARIA VENOSA EIRELI                          |1    |
|DROGARIA MKM EIRELI EPP                         |1    |
|DROGARIA YUMI LTDA ME                           |1    |
|FARMACIA POPULAR DO BRASIL FUNDACAO OSWALDO CRUZ|1    |
|SOU MAIS FARMA COMERCIAL LTDA                   |1    |
|TECNOLOGIA E EMPREENDEDORISMO DA SAUDE LTDA     |1    |
|DROGARIA AKYALI LTDA                            |1    |
|DROGARIA SAO PAULO SA                           |1    |
|DROGARIA BRASIL03 LTDA        

In [7]:
redes.count
redes.where($"count" === 1).count

res6_0: Long = 40L
res6_1: Long = 23L